In [40]:
%run to_import.py
%matplotlib inline

In [2]:
segm_set=range(1,13)
data_folder='../../Data_Raw'
data_small='../../Data_Small_Tables'

In [21]:
#download data
aa_all=range(13)
for s in segm_set:
    aa_all[s]=pd.read_csv(data_folder+'/S'+str(s)+'_scaled.csv',sep='\t')

In [22]:
#download all sequence info
positions=pd.read_csv(data_small+'/positions.csv',sep='\t',index_col=[0])
for s in range(1,13):
    positions.set_value('positions','S'+str(s),int_array_from_string(positions.loc['positions','S'+str(s)]))
wt_seq_all=pd.read_csv(data_small+'/wt_seq.csv',sep='\t',index_col=[0])
wt_data=pd.read_csv(data_small+'/AA_variants.csv',sep='\t',index_col=[0])
d_seqs=func_extract_sequencies_from_alignmnet(data_small+'/rst')

In [29]:
#label nogap,stop,shift,calculate len
for s in range(1,13):
    aa_all[s]['len']=aa_all[s]['aa_seq'].map(lambda x: len(x))
    length=len(wt_seq_all.loc['S'+str(s),'aa_seq'])
    aa_all[s]['nogap']=(1-aa_all[s]['nonsense'])*np.array(map(lambda x: 1 if (x==length) else 0,aa_all[s]['len']))
    aa_all[s]['stop']=map(lambda x: 1 if (x[:-1].count('_')>0) else 0,aa_all[s]['aa_seq'])
    aa_all[s]['shift']=map(lambda x: 1 if (x[-1]=='_') else 0,aa_all[s]['aa_seq'])
    print s, len(aa_all[s]),'nogap',len(aa_all[s][aa_all[s]['nogap']==1]),\
            'stop',len(aa_all[s][aa_all[s]['stop']==1]),\
        'shift',len(aa_all[s][aa_all[s]['shift']==1])
#sum!=all because there are guys with both stops and shifts

1 126087 nogap 61914 stop 39068 shift 61885
2 123219 nogap 84916 stop 26549 shift 36521
3 95082 nogap 72868 stop 11872 shift 21188
4 69220 nogap 65184 stop 3612 shift 3846
5 100509 nogap 75490 stop 14272 shift 23714
6 84037 nogap 65599 stop 11076 shift 16036
7 26377 nogap 16462 stop 7953 shift 9403
8 73806 nogap 60425 stop 10082 shift 12776
9 84245 nogap 81376 stop 1227 shift 2280
10 80233 nogap 64597 stop 2266 shift 14400
11 37422 nogap 34833 stop 1161 shift 2407
12 63825 nogap 58297 stop 3806 shift 5179


In [30]:
#middle ok
for s in range(1,13):
    pos1=int(positions.loc['len1','S'+str(s)])
    pos2=-int(positions.loc['len2','S'+str(s)])
    aa_seq_ref=wt_seq_all.loc['S'+str(s),'aa_seq'][pos1:pos2]
    aa_all[s]['middle']=aa_all[s].aa_seq.map(lambda x: 1 if (hamdist(x[pos1:pos2],aa_seq_ref)==0) else 0)

In [32]:
#mut list
for s in range(1,13):
    wtaa=wt_seq_all.loc['S'+str(s),'aa_seq']
    length=len(wtaa)
    aa_all[s].loc[:,'mut_list']=aa_all[s]['aa_seq'].map(lambda x: make_mut_list(x,wtaa,length))

In [39]:
natural_allele=func_extract_info_about_natural_and_library_variants(wt_data)

In [42]:
for s in range(1,13):
    for tag in ['nat','lib']:
        print s,tag
        indexes=aa_all[s][(pd.notnull(aa_all[s]['mut_list']))&(aa_all[s]['nonsense']==0)&(aa_all[s]['middle']==1)].index
        aa_all[s].loc[:,tag]=map(lambda x:0,range(len(aa_all[s])))
        aa_all[s].loc[indexes,tag]=aa_all[s].mut_list.map(lambda x: is_natural(x,natural_allele.loc[tag,'S'+str(s)]))
    aa_all[s]['nat_lib']=aa_all[s]['nat']*aa_all[s]['lib']

1 nat
1 lib
2 nat
2 lib
3 nat
3 lib
4 nat
4 lib
5 nat
5 lib
6 nat
6 lib
7 nat
7 lib
8 nat
8 lib
9 nat
9 lib
10 nat
10 lib
11 nat
11 lib
12 nat
12 lib


In [48]:
#cut short aa_seq
for s in range(1,13):
    print s
    pos1=int(positions.loc['len1','S'+str(s)])
    pos2=-int(positions.loc['len2','S'+str(s)])
    aa_all[s]['aa_seq_short']=aa_all[s].iloc[:].aa_seq.map(lambda x: x[:pos1]+x[pos2:])
    
    for i in np.sort(list(d_seqs)):
        aa_seq_ref=''.join([d_seqs[i][j] for j in positions.loc['positions','S'+str(s)]])
        aa_all[s].loc[:,'dist_'+i]= map(lambda x:hamdist(x,aa_seq_ref),aa_all[s].loc[:,'aa_seq_short'])

    cols_to_use = ['dist_' +i for i in d_seqs if (i[:4]!='node')]
    aa_all[s]['dist_min_sp']=aa_all[s].loc[:, cols_to_use].min(axis=1)

    cols_to_use = ['dist_' + i for i in d_seqs]
    aa_all[s]['dist_min_all']=aa_all[s].loc[:, cols_to_use].min(axis=1)

1
2
3
4
5
6
7
8
9
10
11
12


In [50]:
for s in range(1,13):
    aa_all[s]['mut_list']=aa_all[s]['mut_list'].map(lambda x: ':'.join(x))

In [55]:
for s in range(1,13):
    aa_all[s].drop(['residuals','aa_seq_short'],axis=1,inplace=True)

In [56]:
aa_all[1].head()

,aa_seq,t0_fr,t1_fr,t2_fr,t0_fr_var,t1_fr_var,t2_fr_var,size,y0,s,y0_std,s_std,nonsense,s_original,s_std_original,len,nogap,stop,shift,middle,mut_list,nat,lib,nat_lib,dist_Agos,dist_Anid,dist_Calb,dist_Cgla,dist_Cgui,dist_Clus,dist_Cpar,dist_Ctro,dist_Dhan,dist_Klac,dist_Kwal,dist_Lelo,dist_Ncra,dist_Sbay,dist_Scas,dist_Scer,dist_Sjap,dist_Sklu,dist_Soct,dist_Spom,dist_Ylip,dist_node_22,dist_node_23,dist_node_24,dist_node_25,dist_node_26,dist_node_27,dist_node_28,dist_node_29,dist_node_30,dist_node_31,dist_node_32,dist_node_33,dist_node_34,dist_node_35,dist_node_36,dist_node_37,dist_node_38,dist_node_39,dist_node_40,dist_node_41,dist_min_sp,dist_min_all
0,AAMGHIKGVARFGSGFAPLDEALSRSVVDF,1.929364,1.653353,0.793621,0.046573,0.054653,0.069003,1,1.863262,0.330914,0.112072,0.016780,0,-0.058103,0.047642,30,1,0,0,1,0A:2M:4H:5I:6K:9A:25S:29F,1,0,0,8,11,6,8,7,7,6,6,7,7,8,6,8,8,7,8,7,7,7,6,8,8,8,8,8,8,8,8,8,8,8,8,7,8,7,7,6,6,6,7,6,6,6
1,AIGAITGLARFGSGFAPLDEALSRAVVDF,2.293858,0.766266,0.000000,0.059952,0.027328,0.000196,1,2.310091,0.000000,0.110412,0.000000,0,-1.142854,0.102815,29,0,0,0,0,,0,0,0,11,13,11,10,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,12,12,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,12,12,10,10
2,AIGDITGVARFGSGFAPLDEALSRAVVDL,3.219973,0.841112,0.064468,0.080148,0.028199,0.003004,1,3.213665,0.000000,0.315899,0.000000,0,-1.327839,0.240556,29,0,0,0,0,,0,0,0,10,12,10,11,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,11,11,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,11,11,10,10
3,AIGDVRGITRFGSGFAPLDEALSRAVVDF,1.195590,0.417492,0.044739,0.027383,0.015429,0.002145,1,1.186915,0.000000,0.141493,0.092004,0,-1.004911,0.268482,29,0,0,0,0,,0,0,0,11,12,11,10,11,11,11,11,11,11,11,11,11,11,10,11,11,11,11,12,12,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,12,12,10,10
4,AIGEFTGIKRFGSGFAPLDEALSRSVVDF,2.228824,0.840634,0.000000,0.057287,0.028192,0.000196,1,2.261129,0.000000,0.164257,0.027362,0,-1.061664,0.141701,29,0,0,0,0,,0,0,0,12,14,12,11,12,12,12,12,12,12,12,12,12,12,12,12,10,12,12,11,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,11,11,10,10


In [57]:
for s in segm_set:
    aa_all[s].to_csv(data_folder+'/S'+str(s)+'_scaled_info.csv',sep='\t',index=False)